by inês <3 doing my best

In [16]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. XGBoost

In [5]:
df = pd.read_pickle("../../extra_features.pkl")
df.xs("HAL", level='Ticker', axis=1)

Price,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,EMA,RSI,Target
Date,,,,,,,,,,,
2010-12-31 00:00:00+00:00,0.997801,0.034612,1.010544,0.015206,1.111599,0.205692,1.306845,0.375537,32.405042,47.505236,-0.004389
2011-01-03 00:00:00+00:00,0.999019,0.003476,1.004139,0.013044,1.105802,0.203945,1.302827,0.358348,32.456154,62.631046,-0.001960
2011-01-04 00:00:00+00:00,0.985056,-0.006349,0.974955,0.022493,1.070865,0.212289,1.263257,0.332691,32.351091,42.035479,-0.029447
2011-01-05 00:00:00+00:00,0.998354,-0.031407,0.977872,-0.001184,1.064969,0.149825,1.258187,0.300775,32.246122,44.496629,-0.003287
2011-01-06 00:00:00+00:00,0.984544,-0.032735,0.961412,-0.031219,1.030942,0.151433,1.219133,0.246995,32.008987,31.477881,-0.030442
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-24 00:00:00+00:00,1.021212,0.038810,1.059307,0.052673,1.006141,-0.122418,1.025281,-0.039847,34.351221,56.022427,0.043343
2024-01-25 00:00:00+00:00,1.002303,0.068611,1.042077,0.104073,1.011902,-0.056044,1.030389,-0.001273,34.670686,63.664102,0.004616
2024-01-26 00:00:00+00:00,1.007111,0.047959,1.036225,0.106583,1.027201,-0.055231,1.045413,-0.020890,35.021279,73.667689,0.014324


When determining the optimal training size for time series forecasting, a key consideration is the balance between the amount of historical data and the ability of the model to generalize effectively. Commonly, a "window size" is chosen, which refers to the number of past observations used to train a model before predicting future points. Research suggests that for time series data, particularly in volatile and unpredictable contexts, using a larger window (e.g., 70% of the data) can improve model performance as it captures enough historical context, but care must be taken to avoid overfitting to past patterns. For example, one approach using a sliding window technique with a 70% training size has been applied successfully, where data is divided into a training set and a test set based on this proportion.

Furthermore, some studies suggest that using cross-validation methods like Time Series Split or Sliding Window cross-validation can provide better insights into model performance across different segments of the data. These methods test the model on multiple, time-sensitive segments and can help determine whether increasing the training size (or adjusting the window) consistently leads to improved predictions.

In terms of practical implementation, experimenting with different window sizes (e.g., 60%, 70%, 80%) and horizons (e.g., predicting 5-day or 10-day ahead) is advised. Testing this across multiple time periods can also help ensure robustness, especially for volatile financial markets or stock prices.

In [ ]:
def time_series_cv(X, y, model, train_size=0.7, horizon=5):
    n = len(y)
    train_len = int(n * train_size)
    
    results = []
    
    for i in range(train_len, n, horizon):
        # Create training and test sets
        train_X, test_X = X[:i], X[i:i+horizon]
        train_y, test_y = y[:i], y[i:i+horizon]
        
        # Fit model (for example, XGBoost)
        model = XGBRegressor()  # Or any other model
        model.fit(train_X, train_y)
        
        # Predict and evaluate performance
        predictions = model.predict(test_X)
        error = mean_squared_error(test_y, predictions)
        
        results.append(error)
    
    return np.mean(results), np.std(results)

1. **Total number of rows**: 3291 rows (representing data from 2011 to 2024).
2. **Training set size**: 70% of the total data. 
   - Training size = 0.70×3291 = 2303.7 ≈ 2304 rows.
3. **Testing set size**: 5 rows for each test (since we want to predict 5 days ahead).
   - Remaining data (for testing):  3291 - 2304 = 987 rows.
   - Number of sets: $$ \text{Number of test sets} = \frac{987}{5} = 197.4 \approx 197 $$ sets.

### Final Calculation:
- **Training set**: 2304 rows (70% of data).
- **Test sets**: 197 sets, each containing 5 days of predictions.


In [17]:
# Hyperparameter tuning using grid search

from sklearn.model_selection import GridSearchCV

param_grid = {

    'learning_rate': [0.01, 0.1, 0.2],

    'max_depth': [3, 5, 7],

    'subsample': [0.8, 0.9, 1.0]

}

grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=3)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

NameError: name 'X_train' is not defined

---

- Chang, V., Xu, Q.A., Chidozie, A. and Wang, H. (2024). Predicting Economic Trends and Stock Market Prices with Deep Learning and Advanced Machine Learning Techniques. Electronics, [online] 13(17), p.3396. doi:https://doi.org/10.3390/electronics13173396.

- Github.io. (2022). Time Series Cross Validation — forecast-tools. [online] Available at: https://tommonks.github.io/forecast-tools/content/03_cross_validation.html [Accessed 5 Dec. 2024].

- Sharma, N. (2024). How to Use XGBoost for Time-Series Forecasting? [online] Analytics Vidhya. Available at: https://www.analyticsvidhya.com/blog/2024/01/xgboost-for-time-series-forecasting/.